In [2]:
# Import necessary libraries
import pandas as pd
import sys
import os

# Add the repo folder to the Python path
repo_path = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Move one directory up
sys.path.append(repo_path)

from utils.binance_data import get_historical_klines, get_usdt_symbols, get_symbols_from_list

# Set options to display all columns and float format to 2 decimal places
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:.2f}')

#### Definimos la funcion para obtener la data historica desde la API de Binance


In [5]:
def collect_binance_data(start_date, end_date, interval="1d", output_file=None, symbols=None):
    """Collect historical data for all USDT pairs from Binance.
    
    Args:
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format
        interval (str, optional): Kline interval. Defaults to "1d"
        output_file (str, optional): Output file path. If None, generates default name
        symbols (list, optional): List of symbols to collect data for. If None, fetches all USDT pairs.
    """
    if output_file is None:
        if symbols is None:
            output_file = f'binance_usdt_pairs_{start_date}_{end_date}_{interval}.csv'

        else:
            output_file = f'binance_{symbols}_{start_date}_{end_date}_{interval}.csv'

    # Get all USDT trading pairs if no symbols are provided
    if symbols is None:
        symbols = get_usdt_symbols()
        print(f"Found {len(symbols)} USDT trading pairs")

    # Initialize an empty DataFrame to store all data
    all_pairs_df = pd.DataFrame()

    # Check if we have existing data
    if os.path.exists(output_file):
        all_pairs_df = pd.read_csv(output_file)
        all_pairs_df['open_time'] = pd.to_datetime(all_pairs_df['open_time'])
        all_pairs_df.set_index(['symbol', 'open_time'], inplace=True)

    # Collect data for each symbol
    for symbol in symbols:
        try:
            print(f"Collecting data for {symbol}...")
            
            last_timestamp = None
            if not all_pairs_df.empty and symbol in all_pairs_df.index.get_level_values('symbol'):
                symbol_data = all_pairs_df.loc[symbol]
                last_timestamp = int(symbol_data.index[-1].timestamp() * 1000)
                print(f"Continuing from {symbol_data.index[-1]} for {symbol}")
            
            df = get_historical_klines(symbol, interval, start_date, end_date, last_timestamp)
            
            if df is not None and not df.empty:
                df['symbol'] = symbol
                
                if not all_pairs_df.empty and symbol in all_pairs_df.index.get_level_values('symbol'):
                    all_pairs_df = all_pairs_df.drop(symbol, level='symbol')
                
                df.reset_index(inplace=True)
                df.set_index(['symbol', 'open_time'], inplace=True)
                
                all_pairs_df = pd.concat([all_pairs_df, df])
                all_pairs_df.to_csv(output_file)
                print(f"Data saved for {symbol}")
                
        except Exception as e:
            print(f"Error collecting data for {symbol}: {str(e)}")

    # Final save and sort
    if not all_pairs_df.empty:
        all_pairs_df.sort_index(inplace=True)
        all_pairs_df.to_csv(os.path.join('..','data',output_file))
        print(f"\nAll data has been saved to {output_file}")

#### Obtenemos los pares en USDT de la API de Binance

In [6]:
# Define parameters
start_date = "2018-12-31"
end_date = "2024-01-01"
interval = "1d"
output_file = f"binance_usdt_pairs_{start_date}_{end_date}_{interval}.csv"

# Check if file exists, if not then collect historical data from Binance API
if not os.path.exists(output_file):
    collect_binance_data(start_date=start_date, end_date=end_date, interval=interval, output_file=None, symbols=None) # 33m 54.4s

In [7]:
data = pd.read_csv("binance_usdt_pairs_2018-12-31_2024-01-01_1d.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426165 entries, 0 to 426164
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   symbol           426165 non-null  object 
 1   open_time        426165 non-null  object 
 2   open             426165 non-null  float64
 3   high             426165 non-null  float64
 4   low              426165 non-null  float64
 5   close            426165 non-null  float64
 6   volume           426165 non-null  float64
 7   close_time       426165 non-null  object 
 8   quote_volume     426165 non-null  float64
 9   num_trades       426165 non-null  int64  
 10  taker_base_vol   426165 non-null  float64
 11  taker_quote_vol  426165 non-null  float64
 12  ignore           426165 non-null  int64  
dtypes: float64(8), int64(2), object(3)
memory usage: 42.3+ MB


In [8]:
# symbol: Trading pair symbol (e.g., "BTCUSDT")
# open_time: Start time of the candlestick period
# open: Opening price of the period
# high: Highest price during the period
# low: Lowest price during the period
# close: Closing price of the period
# volume: Total trading volume in base asset
# close_time: End time of the candlestick period
# quote_volume: Total trading volume in quote asset (USDT)
# num_trades: Number of trades executed in this period
# taker_base_vol: Volume of base asset bought by takers (market orders)
# taker_quote_vol: Volume of quote asset (USDT) bought by takers
# ignore: Unused column from Binance API (can be ignored)

data.head()

,symbol,open_time,open,high,low,close,volume,close_time,quote_volume,num_trades,taker_base_vol,taker_quote_vol,ignore
0,BTCUSDT,2019-01-01,3701.23,3810.16,3642.00,3797.14,23741.69,2019-01-01 23:59:59.999,88149249.09,154227,12919.16,47973435.87,0
1,BTCUSDT,2019-01-02,3796.45,3882.14,3750.45,3858.56,35156.46,2019-01-02 23:59:59.999,133876627.25,218538,17921.60,68277897.66,0
2,BTCUSDT,2019-01-03,3857.57,3862.74,3730.00,3766.78,29406.95,2019-01-03 23:59:59.999,111657372.70,199812,14793.08,56172495.43,0
3,BTCUSDT,2019-01-04,3767.20,3823.64,3703.57,3792.01,29519.55,2019-01-04 23:59:59.999,111034550.64,192232,15579.30,58616203.98,0
4,BTCUSDT,2019-01-05,3790.09,3840.99,3751.00,3770.96,30490.67,2019-01-05 23:59:59.999,115893501.28,203673,14908.91,56667455.39,0


In [9]:
# Convert time columns to datetime
data['open_time'] = pd.to_datetime(data['open_time'])
data['close_time'] = pd.to_datetime(data['close_time'])

data.describe(include='all')

,symbol,open_time,open,high,low,close,volume,close_time,quote_volume,num_trades,taker_base_vol,taker_quote_vol,ignore
count,426165,426165,426165.00,426165.00,426165.00,426165.00,426165.00,426165,426165.00,426165.00,426165.00,426165.00,426165.00
unique,473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NULSUSDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2022-03-19 02:04:48.547393024,217.43,225.13,209.45,217.50,49288942974.14,2022-03-20 02:04:25.605836288,34606491.37,60289.52,24396435277.07,17164146.88,0.00
min,NaN,2019-01-01 00:00:00,0.00,0.00,0.00,0.00,0.00,2019-01-01 23:59:59.999000,0.00,0.00,0.00,0.00,0.00
25%,NaN,2021-05-26 00:00:00,0.08,0.08,0.08,0.08,868804.62,2021-05-26 23:59:59.999000064,859546.32,5720.00,416538.76,410377.34,0.00
50%,NaN,2022-05-01 00:00:00,0.56,0.60,0.53,0.56,6024896.43,2022-05-01 23:59:59.999000064,2977451.84,15285.00,2895703.00,1447073.42,0.00
75%,NaN,2023-03-11 00:00:00,3.82,4.05,3.61,3.81,41682675.44,2023-03-11 23:59:59.999000064,11329649.28,43301.00,20264664.90,5576163.32,0.00
max,NaN,2024-01-01 00:00:00,82875.44,95000.00,66222.40,82885.12,246104304402694.00,2024-01-01 23:59:59.999000,17598561857.08,15920580.00,122064332166743.00,9013969074.85,0.00


#### Obtenemos los pares correspondientes a los pares de trading

In [10]:
# Get unique symbols without USDT
symbols = data['symbol'].unique()
symbols = [s.replace('USDT', '') for s in symbols]

# Generate all possible pairs
pairs = []
for i in range(len(symbols)):
    for j in range(i+1, len(symbols)):
        # Add both combinations for each pair
        pairs.append(symbols[i] + symbols[j])
        pairs.append(symbols[j] + symbols[i])

print(f"Total number of possible trading pairs: {len(pairs)}")
print("\nFirst 10 pairs as example:")
print(pairs[:10])

Total number of possible trading pairs: 223256

First 10 pairs as example:
['BTCETH', 'ETHBTC', 'BTCBNB', 'BNBBTC', 'BTCNEO', 'NEOBTC', 'BTCLTC', 'LTCBTC', 'BTCQTUM', 'QTUMBTC']


In [11]:
len(set(pairs))

223244

In [12]:
existing_pairs = get_symbols_from_list(set(pairs))
len(existing_pairs)

1541

In [13]:
start_date = "2018-12-31"
end_date = "2024-01-01"
interval = "1d"
output_file = f'binance_pairs_{start_date}_{end_date}_{interval}.csv'

# Check if file exists, if not then collect historical data from Binance API
if not os.path.exists(output_file):
    collect_binance_data(start_date=start_date, end_date=end_date, interval=interval,
                        output_file=output_file, symbols=existing_pairs) # 202m 22.2s

Data saved for OPBUSD
Data saved for YFIBNB
Data saved for ALPINEBUSD
Data saved for ACHBTC
Data saved for FETFDUSD
Data saved for INJBUSD
Data saved for BNBBULLBUSD
Data saved for APEBTC
Data saved for ERDUSDC
Data saved for BCHBNB
Data saved for ONTUSDC
Data saved for GTCBTC
Data saved for ETHPAX
Data saved for APEAUD
Data saved for PAXGBUSD
Data saved for DGBBTC
Data saved for BTCDAI
Data saved for AXSBTC
Data saved for FTMEUR
Data saved for ALPINEEUR
Data saved for HARDBNB
Data saved for THETABNB
Data saved for STEEMBUSD
Data saved for APTBUSD
Data saved for LTCGBP
Data saved for RAMPBTC
Data saved for STEEMETH
Data saved for ACMBTC
Data saved for SNTETH
Data saved for SRMBNB
Data saved for ENSBNB
Data saved for THETAETH
Data saved for AAVEBKRW
Data saved for ANKRPAX
Data saved for BCHPAX
Data saved for MAVBTC
Data saved for KP3RBNB
Data saved for VTHOBNB
Data saved for TUSDBUSD
Data saved for NANOBNB
Data saved for MCOBNB
Data saved for XEMBTC
Data saved for ACMBUSD
Data saved for

KeyboardInterrupt: 

In [11]:
data2 = pd.read_csv("binance_pairs_2018-12-31_2024-01-01_1d.csv")
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1183125 entries, 0 to 1183124
Data columns (total 13 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   symbol           1183125 non-null  object 
 1   open_time        1183125 non-null  object 
 2   open             1183125 non-null  float64
 3   high             1183125 non-null  float64
 4   low              1183125 non-null  float64
 5   close            1183125 non-null  float64
 6   volume           1183125 non-null  float64
 7   close_time       1183125 non-null  object 
 8   quote_volume     1183125 non-null  float64
 9   num_trades       1183125 non-null  int64  
 10  taker_base_vol   1183125 non-null  float64
 11  taker_quote_vol  1183125 non-null  float64
 12  ignore           1183125 non-null  int64  
dtypes: float64(8), int64(2), object(3)
memory usage: 117.3+ MB


In [12]:
data2.head()

,symbol,open_time,open,high,low,close,volume,close_time,quote_volume,num_trades,taker_base_vol,taker_quote_vol,ignore
0,1000SATSFDUSD,2023-12-12,0.00,0.00,0.00,0.00,4602853555.00,2023-12-12 23:59:59.999,2525021.75,14569,1910950521.00,1047349.26,0
1,1000SATSFDUSD,2023-12-13,0.00,0.00,0.00,0.00,1881940994.00,2023-12-13 23:59:59.999,929487.73,7854,885572160.00,438627.10,0
2,1000SATSFDUSD,2023-12-14,0.00,0.00,0.00,0.00,4352983453.00,2023-12-14 23:59:59.999,2879741.83,23802,2009894457.00,1336631.79,0
3,1000SATSFDUSD,2023-12-15,0.00,0.00,0.00,0.00,5927834603.00,2023-12-15 23:59:59.999,4821470.11,42360,2461870666.00,2014572.02,0
4,1000SATSFDUSD,2023-12-16,0.00,0.00,0.00,0.00,3099592602.00,2023-12-16 23:59:59.999,2421713.32,13674,1532193355.00,1207083.82,0


In [13]:
# Convert time columns to datetime
data2['open_time'] = pd.to_datetime(data2['open_time'])
data2['close_time'] = pd.to_datetime(data2['close_time'])

data2.describe(include='all')

,symbol,open_time,open,high,low,close,volume,close_time,quote_volume,num_trades,taker_base_vol,taker_quote_vol,ignore
count,1183125,1183125,1183125.00,1183125.00,1183125.00,1183125.00,1183125.00,1183125,1183125.00,1183125.00,1183125.00,1183125.00,1183125.00
unique,1476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,BNBETH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2021-12-18 15:28:43.207607808,4122.45,4231.70,4002.99,4151.60,3083102918.31,2021-12-19 15:27:52.089231360,3513650.57,10269.67,1536250699.22,1745375.21,0.00
min,NaN,2019-01-01 00:00:00,0.00,0.00,0.00,0.00,0.00,2019-01-01 23:59:59.999000,0.00,0.00,0.00,0.00,0.00
25%,NaN,2021-01-29 00:00:00,0.00,0.00,0.00,0.00,39804.91,2021-01-29 23:59:59.999000064,37.61,670.00,18339.60,17.70,0.00
50%,NaN,2022-02-23 00:00:00,0.00,0.00,0.00,0.00,389698.68,2022-02-23 23:59:59.999000064,690.55,2035.00,183563.81,328.13,0.00
75%,NaN,2023-01-01 00:00:00,0.48,0.51,0.46,0.48,3467383.00,2023-01-01 23:59:59.999000064,195538.89,6150.00,1652017.00,92737.26,0.00
max,NaN,2024-01-01 00:00:00,47615392.00,48113580.00,45021551.00,47615392.00,50522076714408.00,2024-01-01 23:59:59.999000,7278601783.28,20117333.00,25171293531170.00,3583831756.36,0.00
